# CSK constellation optimization

* $\alpha \in \mathbb{R}^{N\times 3}$ is the set of input points, N vectors of length 3 (r,g,b)
* $H \in \mathbb{R}^{3\times 3}$ is the transformation matrix


The problem to solve is:
$$
\begin{aligned}
\max \min_{\alpha} \quad & ||\alpha H_{i,:} - \alpha H_{j,:}||_{2}^{2}, i=1...N, j=1...N, i\ne j\\
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
\end{aligned}
$$


This $\max \min$ problem can be rewritten as:

$$
\begin{aligned}
\max_{\alpha} \quad & Z\\ 
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
  & Z \le ||\alpha H_{1,:} - \alpha H_{2,:}||_{2}^{2}\\
  & Z \le ||\alpha H_{1,:} - \alpha H_{3,:}||_{2}^{2}\\
  & \vdots
\end{aligned}
$$

---
Install and import required packages

In [1]:
using Pkg
Pkg.add("JuMP")        # to express optimization problems
Pkg.add("Ipopt")       # solver
Pkg.add("PlotlyJS")    # for plotting
Pkg.add("DataFrames")
Pkg.add("LinearAlgebra")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Check packages versions (for reference)

In [2]:

Pkg.status()

Status `~/.julia/environments/v1.9/Project.toml`
  [336ed68f] CSV v0.10.11
  [a93c6f00] DataFrames v1.6.1
  [7073ff75] IJulia v1.24.2
  [b6b21f68] Ipopt v1.4.2
⌃ [4076af6c] JuMP v1.15.0
  [f0f68f2c] PlotlyJS v0.18.10
  [0f1e0344] WebIO v0.8.21
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
using JuMP
using Ipopt
using PlotlyJS
using LinearAlgebra
using DataFrames

WebIO._IJuliaInit()

In [4]:
manhattan(x,y; M=3) = sum((abs(x[i]-y[i]) for i in 1:M))
euclid(x,y; M=3) = sqrt(sum((x[i]-y[i])^2 for i in 1:M))

# The Euclidean norm can be used with a small trick: a) remove the sqrt, and
# modify the Z < dist constraint t Z*Z < dist (see below in the model definition)
euclid2(x,y; M=3) = sum((x[i]-y[i])^2 for i in 1:M)


euclid2 (generic function with 1 method)

In [5]:

function opt(N::Int64=4; H::Union{Vector{Matrix{Float64}}, Nothing}=nothing, x0::Union{Matrix{Float64},Nothing}=nothing, norm=manhattan, M=3, rewrite_maxmin=true, print_level=0)
	if H == nothing
		# Defaults to a single identity matrix
		H = [I(M)]
	end

	if x0 == nothing
		# Defaults to a random matrix with values between 0 and 1
		x0 = rand(N, M)
	end

	error = false

	model = Model(Ipopt.Optimizer)
	set_attribute(model, "print_level", print_level)  # up to 5
	@variable(model, ɑ[1:N, 1:M]  >= 0)
	@constraint(model, sum(ɑ, dims=2) .- 1 .== 0)


	set_start_value.(ɑ, x0)

	dist_ = []
	for h in H
		ɑH = ɑ*h  # define output constellation

		# calculate pairwise distances
		dist_ = vcat(dist_, [norm(ɑH[i,:], ɑH[j,:], M=3) for i in 1:N for j in 1:N if i != j])
	end

	Nd = size(dist_)[1]
	@expression(model, dist[i=1:Nd], dist_[i])
	if rewrite_maxmin
		# uses the rewriting of the maxmin
		@variable(model, Z >= 0)
		if norm == euclid2
			@constraint(model, Z^2 .<= dist) # if euclid 2
		else
			@constraint(model, Z .<= dist) # if euclid 2
		end
		@objective(model, Max, Z)
	else
		# this approch is currently not working. TODO: investigate
		f(args...) = min(args...)
		@objective(model, Max, f(dist...))
	end
	optimize!(model)

	if termination_status(model) != MOI.LOCALLY_SOLVED
		error = true
	end
	
	return error, value(Z), x0, value.(ɑ)
end

opt (generic function with 2 methods)

In [6]:
H1 = 55.3e-3*[1 0.042  0.030; 0.194  0.665  0.277;0.009 0.084 0.421]
H2 = 56.4e-3*[1 0.03  0.024; 0.195  0.623  0.258;0.008 0.077 0.414]
H = [H1, H2]



2-element Vector{Matrix{Float64}}:
 [0.0553 0.0023226 0.001659; 0.0107282 0.0367745 0.015318100000000001; 0.0004977 0.0046452 0.0232813]
 [0.0564 0.001692 0.0013536; 0.010998 0.0351372 0.0145512; 0.0004512 0.0043428 0.023349599999999998]

In [7]:
# Function to perform multiple experiments with random initialization
# The strategy can be changed of course

function random_exp(N, norm, n_exp=40)
	Z_best = 0
	alpha_best = 0
	x0_best = 0
	for i in 1:n_exp
		# uses a random initialization
		# H defaults to [I]: a single identity matrix
		error, Z, x0, ɑ = opt(N, norm=euclid2)
		if error == false
			if Z > Z_best
				Z_best = Z
				alpha_best = ɑ
				x0_best = x0
			end
		end
	end
	return Z_best, alpha_best, x0_best
end;

In [8]:
N = 16
n_exp=40
Z_man, alpha_man, x0_man = random_exp(N, manhattan, n_exp)
Z_eucl, alpha_eucl, x0_eucl = random_exp(N, euclid2, n_exp)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



(0.30579151415584105, [0.3940494089843711 0.6059505859112365 5.104392520652491e-9; 0.24845608510620454 0.5030879167204805 0.248455998173315; … ; 6.1145806926428e-9 0.6486818115087309 0.3513181823766884; 0.4090233766803743 0.04099364735755998 0.5499829759620657], [0.5154195990911186 0.7575473948480989 0.06458023051489736; 0.6660387410896558 0.7811981642357159 0.30791657383895576; … ; 0.08808682104360355 0.4911132677751767 0.5686473187926145; 0.8438134590165258 0.021023332635996872 0.7245036626051599])

In [9]:
exps = [alpha_man, alpha_eucl]
exp_names = ["Manhattan", "Euclid2"]

2-element Vector{String}:
 "Manhattan"
 "Euclid2"

In [10]:
df_out = DataFrame()
for i in 1:size(exps)[1]
	output = exps[i] 
	df = DataFrame(output, :auto)
	df[!, :name] .= exp_names[i]
	df_out = vcat(df_out, df)
end

In [11]:
plot(df_out, x=:x1, y=:x2, z=:x3, type="scatter3d", mode="markers", color=:name)

data: [
  "scatter3d with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, yaxis, and z",
  "scatter3d with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, yaxis, and z"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"